In [192]:
import pandas as pd
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package wordnet to /home/div/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/div/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [193]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import ngrams
from difflib import SequenceMatcher
import math

In [194]:
data_frame = pd.read_csv('data.csv')
data_top = data_frame.columns

In [195]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()
features = []
features_list = []
for feature in data_top:
    words_list = feature.split('_')
    feature_words = []
    for word in words_list:
        if word not in stop_words:
            feature_words.append(lemmatizer.lemmatize(word))
    features.append('_'.join(feature_words))
    features_list.append(feature_words)

In [196]:
colum_len = len(data_top)
x_data =  data_frame.iloc[:,:132]
y_data =  data_frame.iloc[:,132:133]

In [197]:
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data)

In [199]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(x_train,y_train)
y_predict = rf_classifier.predict(x_test)
score = accuracy_score(y_test,y_predict)
print(score)

1.0


/home/div/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/div/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [191]:
speech_test = "my child is fever very high"

ngram_list = []

all_ngrams = ngrams(speech_test.split(),1)
for ngram in all_ngrams:
    lowered_ngram_tokens = map(lambda token: token.lower(), ngram)
    if any(token not in stop_words for token in lowered_ngram_tokens):
         ngram_list.append(' '.join(ngram))

all_ngrams = ngrams(speech_test.split(),2)
for ngram in all_ngrams:
    lowered_ngram_tokens = map(lambda token: token.lower(), ngram)
    if any(token not in stop_words for token in lowered_ngram_tokens):
         ngram_list.append(' '.join(ngram))

all_ngrams = ngrams(speech_test.split(),3)
for ngram in all_ngrams:
    lowered_ngram_tokens = map(lambda token: token.lower(), ngram)
    if any(token not in stop_words for token in lowered_ngram_tokens):
         ngram_list.append(' '.join(ngram))

train_list = []
for element in ngram_list:
    temp = element.split(' ')
    temp2 = []
    for word in temp:
        temp2.append(lemmatizer.lemmatize(word))
    train_list.append(temp)
#train_list
#features_list
test_data = []
for i in range(0,len(features_list)-1):
    test_data.append(0)
    
for word in train_list:
    for feat in features_list:
        word1 = " ".join(word)
        word2 = " ".join(feat)
        cosine_similarity = get_cosine(text_to_vector(word1),text_to_vector(word2))
        if cosine_similarity > 0.75:
            index = features_list.index(feat)
            test_data[index] = 1

disease_predict = rf_classifier.predict([test_data])
prob = rf_classifier.predict_proba([test_data])
print(disease_predict,prob)
for i in range(0,len(prob[0])):
    if prob[0][i] > 0.0:
         print(rf_classifier.classes_[i],prob[0][i])

['AIDS'] [[0.  0.5 0.  0.  0.  0.  0.  0.  0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.1 0.1 0.1 0.  0.  0.1 0.  0.  0.
  0.  0.  0.  0.  0. ]]
AIDS 0.5
Chicken pox 0.1
Impetigo 0.1
Jaundice 0.1
Malaria 0.1
Paralysis (brain hemorrhage) 0.1


In [200]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+') 

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)